Intially I was going to try word2vec, but it was more complicated to figure out for me than tfidf and I just ran out of time.

In [0]:
pip install word2vec --quiet

I imported what looks like more than necessary, but I tried a lot of things in here.

In [0]:
import pandas as pd
import numpy as np
import random
import json
import re
# import word2vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

In [0]:
df = pd.read_csv('/content/drive/My Drive/Colab Datasets/COVID Chat Bot/covid_df.csv', header=0, names=['questions', 'answers'])

Found this pos_tag function because the lemmatizer was only lemmatizing nouns.

In [0]:
def get_wordnet_pos(word):
  #Map POS tag to first character lemmatize() accepts
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {"J": wordnet.ADJ,
              "N": wordnet.NOUN,
              "V": wordnet.VERB,
              "R": wordnet.ADV}

  return tag_dict.get(tag, wordnet.NOUN)

This the preprocess function we used in multiple places throughout the program. I ultimately decided against using the stemmer because our data just isn't large enough to be worried about computation cost.

In [0]:
def preprocess(docs):
  lemmer = WordNetLemmatizer()
  stemmer = SnowballStemmer('english')
  prepd = []

  for doc in docs:
    tokenized = word_tokenize(doc)

    cleaned = [lemmer.lemmatize(token.lower(), get_wordnet_pos(token))
               for token in tokenized
               if token.lower() not in stopwords.words('english')
               if token.isalpha()]

    untokenized = ' '.join(cleaned)
    prepd.append(untokenized)

  return prepd

Testing the lemmatizer. It did not lemmatize any of my verbs, and it wasn't till I tried a noun that it finally worked. That's when I realized I needed a tagger.

In [0]:
lemmer = WordNetLemmatizer()
lemmer.lemmatize('bosses')

'boss'

Processing the question bank.

In [0]:
lemm_stemm = preprocess(df['questions'])

Making sure it worked.

In [0]:
lemm_stemm

My first vectorizer was the CountVec. It had good results early, but broke down in stress tests.

In [0]:
vectorizer = CountVectorizer()
count_vec = vectorizer.fit_transform(lemm_stemm).toarray()

The first bot. It perfomed ok, but wasn't all that. Good ideas formed here, though.

In [0]:
# def COVIDbot(user_response):
#   text = vectorizer.transform([user_response]).toarray()
#   df['similarity'] = cosine_similarity(count_vec, text)
#   scores = df.sort_values(['similarity'], ascending=False) #.iloc[0]['answers']
#   if scores.iloc[0]['similarity'] >= 0.8:
#     print(scores.iloc[0])
#     return scores.iloc[0]['answers']
#   else:
#     first = scores.iloc[0]['questions']
#     second = scores.iloc[1]['questions']
#     print(f'I do not understand that question.\nDo you mean "{first}", or "{second}"?\nPlease type or copy and paste the question that is most like yours.')

Second bot iteration. This performed better, but had some quirks due to the print statement and empty return that I wouldn't find until later. We also officially switch from the CountVec to the TF-IDF, so obviously this cell was moved up from its previous position. I needed to see both COVID and COVID2 at the same time for some code comparison.

In [0]:
# def COVID2bot(user_response):
#   text = model.transform([user_response]).toarray()
#   df['similarity'] = cosine_similarity(tfid, text)
#   scores = df.sort_values(['similarity'], ascending=False) #.iloc[0]['answers']
#   if scores.iloc[0]['similarity'] >= 0.6:
#     print(scores.iloc[0]['answers'])
#     return
#   else:
#     first = scores.iloc[0]['questions']
#     second = scores.iloc[1]['questions']
#     user_input = input(f'I do not understand that question.\nDo you mean "{first}", or "{second}"?\nPlease type "1", "2", or "no".\n')
#     if user_input == '1':
#       print(scores.iloc[0]['answers'])
#       return
#     elif user_input == '2':
#       print(scores.iloc[1]['answers'])
#       return
#     else:
#       print('Please ask another question.')
#       return


Bot testing!

In [0]:
COVIDbot('what is coronavirus?')

questions                                What is a coronavirus?
answers       Coronaviruses are a large family of viruses wh...
similarity                                                    1
Name: 134, dtype: object


'Coronaviruses are a large family of viruses which may cause illness in animals or humans. In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS). The most recently discovered coronavirus causes coronavirus disease COVID-19.'

Stress testing!

In [0]:
COVIDbot('What is your name?')

I do not understand that question.
Do you mean "What is a novel coronavirus?", or "Should I worry about COVID-19?"?
Please type or copy and paste the question that is most like yours.


Here we switch to the TF-IDF

In [0]:
model = TfidfVectorizer()

In [0]:
tfid = model.fit_transform(lemm_stemm).toarray()

The new and improved bot! I added a third option. I had started with a similarity threshhold of .6 but it gave...weird answers. The .8 has worked much better.

In [0]:
def COVID2bot(user_response):
  text = model.transform([user_response]).toarray()
  df['similarity'] = cosine_similarity(tfid, text)
  scores = df.sort_values(['similarity'], ascending=False) #.iloc[0]['answers']
  if scores.iloc[0]['similarity'] >= 0.8:
    return (scores.iloc[0]['answers'])
  else:
    first = scores.iloc[0]['questions']
    second = scores.iloc[1]['questions']
    third = scores.iloc[2]['questions']
    user_input = input(
        f'I do not understand that question.\nDo you mean "{first}", "{second}", or "{third}"\nPlease type 1, 2, 3, or hit return to ask a different question.\n')
    if user_input == '1':
      return (scores.iloc[0]['answers'])
    elif user_input == '2':
      return (scores.iloc[1]['answers'])
    elif user_input == '3':
      return (scores.iloc[2]['answers'])
    else:
      return ('Please ask another question.')


Stress testing!

In [0]:
COVID2bot('What is a zebra?')

Welcome bot

In [0]:
welcome_input = ("hello", "hi", "greetings", "sup", "what's up","hey",)
welcome_response = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad you are talking to me!"]
def welcome(user_response):
    for word in user_response.split():
        if word.lower() in welcome_input:
            return random.choice(welcome_response)

Chatbot V2.1

In [0]:
flag=True
print("Greetings! I am a chatbot, and I will try to answer your questions about COVID-19. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response = user_response.lower()
    if(user_response not in ['bye','shutdown','exit', 'quit']):
        if(user_response=='thanks' or user_response=='thank you'):
            flag=False
            print("Chatbot : You are welcome..")
        else:
            if(welcome(user_response)!=None):
                print("Chatbot : "+welcome(user_response))
            else:
                print("Chatbot : ",end="")
                print(COVID2bot(user_response))
    else:
        flag=False
        print("Chatbot : Bye!!! ")

Greetings! I am a chatbot, and I will try to answer your questions about COVID-19. If you want to exit, type Bye!
can i catch corona from my pet?
Chatbot : While there has been one instance of a dog being infected in Hong Kong, to date, there is no evidence that a dog, cat or any pet can transmit COVID-19. COVID-19 is mainly spread through droplets produced when an infected person coughs, sneezes, or speaks. To protect yourself, clean your hands frequently and thoroughly. WHO continues to monitor the latest research on this and other COVID-19 topics and will update as new findings are available.
can i catch corona from my cat?
Chatbot : I do not understand that question.
Do you mean "Can I catch COVID-19 from my pet?", "How likely am I to catch COVID-19?", or "Is it true that animals, like dogs, cats, and cattle, get their own types of coronavirus?"
Please type 1, 2, 3, or hit return to ask a different question.

Please ask another question.
can i catch corona from my dog?
Chatbot : I 

Bot test code without all the bells and whistles.

In [0]:
user_input = input('What is your question?\n')
while user_input.lower() != 'quit':
  COVID2bot(user_input)
  user_input = input('What is your question?\n')